In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/tmp/ipykernel_109075/1425855459.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-13 23:00:01.881895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 23:00:01.881966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 23:00:01.882752: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable

In [2]:
game_threshold = 4 # minimum number of games to be in the dataset
test_size = 0.01 # 20% of the dataset is used for testing
ModelHasBot = True #  if True, the bot is considered as a player - to consider bot statistics in games 2vs3 or 3vs4
timeStart ='' #'2023-03-01' # start date of the dataset

# 'Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран'
present_players = ['Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун','Сеньор','Юран']

# Data preprocessing

In [3]:
dataset = pd.read_csv('data.csv') 
columnsPlayers = ["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]
columnsTeam1 = ["Player1","Player2","Player3","Player4"]
columnsTeam2 = ["Player5","Player6","Player7","Player8"]

In [4]:
dataset["Map"] = dataset["Map"].str.lower() # convert all maps to lower case
dataset['Date'] = pd.to_datetime(dataset['Date'],format='%d.%m.%Y') # convert date to datetime
dataset=dataset.drop_duplicates(subset=['Date','Map']) # drop duplicates with same date and map
dataset.to_csv('new_data.csv',index=False) # save the new dataset
dataset.drop(columns=['link','Recorder','vs'],   inplace=True) # drop columns that are not needed

#  drop entries which were played before the start date
if timeStart != '':
    dataset = dataset[dataset['Date'] >= timeStart]
dataset


,Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8
0,2024-03-09,ливень,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот
3,2024-03-03,кровавая жатва,Данил,Вадим,Сеньор,Савва,Гриша,Миша,Ден,Бот
6,2024-03-03,переход,Вадим,Сеньор,Юран,Гриша,Ден,Данил,Савва,Миша
8,2024-03-02,мрачный карнавал,Данил,Гриша,Сеньор,Савва,Вадим,Миша,Ваня,Бот
11,2024-02-25,холодный ручей,Ден,Вадим,Гриша,Бот,Юран,Сеньор,Ваня,Бот
...,...,...,...,...,...,...,...,...,...,...
122,2023-03-10,смерть в воздухе,Ден,Миша,Вадим,Юран,Данил,Ваня,Гриша,Савва
125,2023-03-05,жертва,Сеньор,Вадим,Гриша,Бот,Данил,Ваня,Ондрей,Бот
127,2023-03-05,мрачный карнавал,Сеньор,Гриша,Ваня,Бот,Данил,Вадим,Ондрей,Бот
129,2023-03-04,вымерший центр,Гриша,Данил,Ден,Джун,Ваня,Сеньор,Миша,Бот


### Add column with number of maps in each campaign

In [ ]:
map_weights = {
'ливень':5,
'кровавая жатва':5, 
'переход':3,
'мрачный карнавал':5,
'холодный ручей':4,
'похоронный звон':5,
'приход':5,
'смерть в воздухе':5,
'нет милосердию':5,
'вымерший центр':4,
'болотная лихорадка':4,
'последний рубеж':2,
'жертва':3,
'роковой полет':5}

def map_weight(map):
    return map_weights[map]

dataset["Map_weight"] = dataset["Map"].apply(map_weight)
dataset["Map_weight"] = dataset["Map_weight"]/ dataset["Map_weight"].max()# scale weight

#### Unique players and cleaning those with small statistics

In [ ]:
# unique players from all 8 columns ["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]
players = np.unique(dataset[columnsPlayers].values)
players = pd.DataFrame(players, columns=["Player"])
# filter players that have played more than 'game_threshold' games
players = players[players["Player"].map(dataset[columnsPlayers].stack().value_counts()) >= game_threshold]
players = players["Player"].values

# remove rows with players not in the list
dataset = dataset[dataset[columnsPlayers].isin(players).all(axis=1)]

if not ModelHasBot:
    players = players[players != 'Бот']
players

In [ ]:
# randomly assign 1 or 2 to TeamWon column
dataset["TeamWon"] = np.random.randint(1, 3, dataset.shape[0]) # 1 or 2

In [ ]:
# Players from "Team 1" are Player1, Player2, Player3, Player4
# Players from "Team 2" are Player5, Player6, Player7, Player8
# TeamWon column is the team that won the game
#  assign to player in which team he was
for player in players:
    dataset[player] = 0
    dataset.loc[(dataset['Player1'] == player) | (dataset['Player2'] == player) | (dataset['Player3'] == player) | (dataset['Player4'] == player), player] = 1
    dataset.loc[(dataset['Player5'] == player) | (dataset['Player6'] == player) | (dataset['Player7'] == player) | (dataset['Player8'] == player), player] = 2
    if player=="Bot":
        dataset.loc[(dataset['Player4'] == "Bot") & (dataset['Player8'] == "Bot"), 'Bot'] = 0 # remove bot from statistics of 3 vs 3


In [ ]:
#  add data with swapped players to lower values of absent players
swapped = dataset.copy()
swapped[columnsTeam1+columnsTeam2] = swapped[columnsTeam2+columnsTeam1]
dataset = pd.concat([dataset, swapped])

## Train the model

In [ ]:
# split the dataset into train and test
train, test = train_test_split(dataset,  test_size=test_size, random_state=42)

# select the columns to predict
y_train = train['TeamWon']-1  # 0 or 1
y_test = test['TeamWon']-1 # 0 or 1

# select the columns to train the model
X_train = train[players]
w_train = train["Map_weight"]
X_test = test[players]
w_test = test["Map_weight"]

#  scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#  create a model
model = keras.Sequential([
    keras.layers.Dense(100, input_shape=(len(players),), activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
            #   weighted_metrics=['accuracy']
              )
# train the model
model.fit(X_train, y_train, sample_weight=w_train, epochs=10)
# test the model
print ("")
print ("Test")
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

In [ ]:
all_results = []
# add bots to the present players if there are less than 8 players
while len(present_players) < 8:
      present_players = present_players + ["Бот"]
print ('present_players are', present_players)
for i in range(0, len(present_players)):
    for j in range(i+1, len(present_players)):
        for k in range(j+1, len(present_players)):
            for l in range(k+1, len(present_players)):
                                team1 = [present_players[i], present_players[j], present_players[k], present_players[l]]
                                # subtract team 1 player form list of present players, "Бот" can be used many times
                                team2 = [x for x in present_players if x not in team1]
                                # if team 2 has less than 4 players, add "Бот" to team 2
                                while len(team2) < 4:
                                    team2.append("Бот")
                                row = pd.DataFrame([team1 + team2], columns=columnsPlayers)
                                 # assign 1 or 2 to each player based on the team
                                for player in players:
                                    row[player] = 0
                                    if player in team1:
                                        row[player] = 1
                                    if player in team2:
                                        row[player] = 2
                                # scale the data
                                row = scaler.transform(row[players])
                                # predict the result
                                result = model.predict(row, verbose=0)
                                all_results.append((team1, team2, result))
# sort the results
all_results.sort(key=lambda x: x[2], reverse=True)

In [ ]:
# print the results
for i in range(0, len(all_results)):
    print(all_results[i][0], "vs", all_results[i][1], ":", all_results[i][2][0][0])

In [ ]:
#  take the middle of the list
middle = len(all_results) // 2
print ("best team by median:")
print(all_results[middle][0], "vs", all_results[middle][1], "with output=", all_results[middle][2][0][0])

In [ ]:
# take closest to 50% result
closest = min(all_results, key=lambda x: abs(x[2] - 0.5))
print ("best team by model 50% result:")
print(closest[0], "vs", closest[1], "with output=", closest[2][0][0])